#### Check the current working directory and shift to main  one

In [104]:
import os

In [105]:
pwd

'/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text _Summarization/Text_Summarization_Project'

In [42]:
cd ..

/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text _Summarization


##### NOTE : code should be done In The Main /Text_Summarization_Project Directory

In [100]:
pwd

'/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text _Summarization/Text_Summarization_Project'

## Workflow

1. Update config.yaml 
2. Update params.yaml
3. Update entity 
4. Update Configuration manager in SRC config
5. Update the components
6. Update the pipeline
7. Update main.py
8. Update app.py (very last this will be done)


In [106]:
#03.Update Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [107]:
#04. Update Configuration manager in SRC config

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [108]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [109]:
#05. Update the components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [110]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [111]:
#06. Update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-26 02:29:37,781: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-26 02:29:37,782: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-26 02:29:37,783: INFO: common: created directory at: artifacts]
[2025-01-26 02:29:37,784: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-26 02:29:38,372: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 26DA:317608:4B150E:4DF847:67957818
Accept-Ranges: bytes
Date: Sun, 26 Jan 2025 01:29:38 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-etou8220124-FRA
X-C